In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
from scipy.stats import t, chi2_contingency

In [ ]:
# download file A/B/C/D
url = "https://drive.google.com/file/d/12jDqkC3N6nH9wBn-8XiHVsEaDgY5RjZW/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
result_a = pd.read_csv(path)

url = "https://drive.google.com/file/d/1ijRU-kiAU8ZnGk-pXjpa-l8oPdmdULfz/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
result_b = pd.read_csv(path)

url = "https://drive.google.com/file/d/1RnOMYsulaXQ8ZwGIqwAVANe_FZYAlM2d/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
result_c = pd.read_csv(path)

url = "https://drive.google.com/file/d/1g-ZaPkAOw3Uh5uxPJgamqw2RlGuqZqZJ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
result_d = pd.read_csv(path)

In [ ]:
a_result = result_a.drop('Snapshot information', axis = 1).copy()
b_result = result_b.drop('Snapshot information', axis = 1).copy()
c_result = result_c.drop('Snapshot information', axis = 1).copy()
d_result = result_d.drop('Snapshot information', axis = 1).copy()

#Exploring the results

In [ ]:
a_result.info()
b_result.info()
c_result.info()
d_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Element ID  57 non-null     int64 
 1   Tag name    57 non-null     object
 2   Name        57 non-null     object
 3   No. clicks  57 non-null     int64 
 4   Visible?    57 non-null     bool  
dtypes: bool(1), int64(2), object(2)
memory usage: 2.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Element ID  57 non-null     int64 
 1   Tag name    57 non-null     object
 2   Name        57 non-null     object
 3   No. clicks  57 non-null     int64 
 4   Visible?    57 non-null     bool  
dtypes: bool(1), int64(2), object(2)
memory usage: 2.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 5 columns):
 #   Column     

In [ ]:
a_result.head(10)

,Element ID,Tag name,Name,No. clicks,Visible?
0,48,h1,ENIAC,269,True
1,25,div,mySidebar,309,True
2,4,a,Mac,279,True
3,69,a,iPhone,246,True
4,105,a,Accessories,1235,True
5,36,a,Chargers & Cables,1261,False
6,99,a,iPhone Accessories,1226,False
7,68,a,Watch Accessories,1261,False
8,13,a,Mac Accessories,1308,False
9,15,a,AirTag,206,False


In [ ]:
a_result['version'] = 'A'
b_result['version'] = 'B'
c_result['version'] = 'C'
d_result['version'] = 'D'

In [ ]:
total_results = pd.concat([a_result, b_result, c_result, d_result]).reset_index()
total_results = total_results.drop('index', axis=1)
total_results

,Element ID,Tag name,Name,No. clicks,Visible?,version
0,48,h1,ENIAC,269,True,A
1,25,div,mySidebar,309,True,A
2,4,a,Mac,279,True,A
3,69,a,iPhone,246,True,A
4,105,a,Accessories,1235,True,A
...,...,...,...,...,...,...
223,96,a,Instagram,274,True,D
224,114,a,Snapchat,252,True,D
225,40,a,Pinterest,300,True,D
226,26,a,Twitter,265,True,D


In [ ]:
total_clicks = total_results.loc[total_results['Name'].isin({'SHOP NOW', 'SEE DEALS'}), :]
total_clicks

,Element ID,Tag name,Name,No. clicks,Visible?,version
21,106,a,SHOP NOW,512,True,A
78,126,a,SHOP NOW,281,True,B
135,127,a,SEE DEALS,527,True,C
192,128,a,SEE DEALS,193,True,D


In [ ]:
total_clicks = total_clicks.rename(columns = {'No. clicks': 'clicks'})
total_visits = [25326, 24747, 24876, 25233]
total_clicks['visits'] =  total_visits
total_clicks['no_click'] = total_clicks['visits'] - total_clicks['clicks']
total_clicks['ctr'] = total_clicks['clicks']/total_clicks['visits']
total_clicks = total_clicks.set_index(pd.Index(['A', 'B', 'C', 'D']))
total_clicks

,Element ID,Tag name,Name,clicks,Visible?,version,visits,no_click,ctr
A,106,a,SHOP NOW,512,True,A,25326,24814,0.020216
B,126,a,SHOP NOW,281,True,B,24747,24466,0.011355
C,127,a,SEE DEALS,527,True,C,24876,24349,0.021185
D,128,a,SEE DEALS,193,True,D,25233,25040,0.007649


In [ ]:
total_clicks['ctr'].sort_values(ascending = False)

,ctr
C,0.021185
A,0.020216
B,0.011355
D,0.007649


In [ ]:
test_results = total_clicks[['version', 'clicks', 'no_click']]
test_results =  test_results.set_index('version').transpose()
test_results

version,A,B,C,D
clicks,512,281,527,193
no_click,24814,24466,24349,25040


#Calculate Chi square

In [ ]:
from scipy import stats
en_chisq, en_p_value, en_df, en_expected = stats.chi2_contingency(test_results)

In [ ]:
en_expected_table = pd.DataFrame(en_expected)

In [ ]:
print(f""" chi square = {en_chisq}
      p value = {en_p_value}
      degree of freedome = {en_df}
      expected values = \n{en_expected_table}""")

 chi square = 224.01877488058412
      p value = 2.7161216607868712e-48
      degree of freedome = 3
      expected values = 
              0           1             2             3
0    382.486255    373.7419    375.690124    381.081721
1  24943.513745  24373.2581  24500.309876  24851.918279


In [ ]:
en_alpha = 0.05

In [ ]:
if en_p_value < en_alpha:
  print("reject the null hypothesis: the 4 versions performs differently with statiscally significant (95% confidence)")
else:
  print("do not reject the null hypothesis: there is no sufficint difference in 4 versions")

reject the null hypothesis: the 4 versions performs differently with statiscally significant (95% confidence)


#Post-Hoc Test

In [ ]:
ctr = total_clicks.loc[total_clicks['version'] == 'A', 'ctr']
ctr = float(ctr.iloc[0])

In [ ]:
ctr

0.02021637842533365

In [ ]:
new_alpha = 0.05/6    # 6 is the number of pairs we will do the post-hoc

In [ ]:
def post_hoc(test_results, version_1, version_2, alpha, total_clicks):
  paired_table = test_results[[version_1, version_2]]
  en_chisq, en_p_value, en_df, en_expected = stats.chi2_contingency(paired_table)

  print(f"p-value is {en_p_value} and alpha is {alpha}")
  if en_p_value < alpha:
    print(f"reject the null hypothesis: {version_1} and {version_2} perform significantly different")
  else:
    print(f"do not reject the null hypothesis: there is no sufficint difference in {version_1} and {version_2} versions")

  ctr_1 = total_clicks.loc[total_clicks['version'] == version_1, 'ctr']
  ctr_1 = float(ctr_1.iloc[0])
  ctr_2 = total_clicks.loc[total_clicks['version'] == version_2, 'ctr']
  ctr_2 = float(ctr_2.iloc[0])

  if ctr_1 > ctr_2:
    print(f"version {version_1}: {ctr_1} CTR rate is higher than {version_2}: {ctr_2}")
  elif ctr_2 > ctr_1:
    print(f"version {version_2}: {ctr_2} CTR rate is higher than {version_1}: {ctr_1}")
  else:
    print("CTR inconclusive")

  print("""
  =====================================================================================

  """)



In [ ]:
#Compare version A and B
post_hoc(test_results, 'A', 'B', new_alpha, total_clicks)

#Compare version A and B
post_hoc(test_results, 'A', 'C', new_alpha, total_clicks)

#Compare version A and B
post_hoc(test_results, 'A', 'D', new_alpha, total_clicks)

#Compare version A and B
post_hoc(test_results, 'B', 'C', new_alpha, total_clicks)

#Compare version A and B
post_hoc(test_results, 'B', 'D', new_alpha, total_clicks)

#Compare version A and B
post_hoc(test_results, 'C', 'D', new_alpha, total_clicks)

p-value is 2.6730880539675666e-15 and alpha is 0.008333333333333333
reject the null hypothesis: A and B perform significantly different
version A: 0.02021637842533365 CTR rate is higher than B: 0.01135491170646947

  
  
p-value is 0.464842123017306 and alpha is 0.008333333333333333
do not reject the null hypothesis: there is no sufficint difference in A and C versions
version C: 0.0211850779868146 CTR rate is higher than A: 0.02021637842533365

  
  
p-value is 3.0808983951735623e-33 and alpha is 0.008333333333333333
reject the null hypothesis: A and D perform significantly different
version A: 0.02021637842533365 CTR rate is higher than D: 0.007648713985653708

  
  
p-value is 6.955480170773988e-18 and alpha is 0.008333333333333333
reject the null hypothesis: B and C perform significantly different
version C: 0.0211850779868146 CTR rate is higher than B: 0.01135491170646947

  
  
p-value is 2.3573374085176145e-05 and alpha is 0.008333333333333333
reject the null hypothesis: B and D

#Additional Analyzation